In [16]:
import numpy as np
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import estimator as est
rng = np.random.default_rng(seed=315)
import cProfile
import re


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# set sim_name
sim_name='sim_pair_30' # other version will be 'diffSeed'
sim_dir='central'
snap_num = 6
# set phase
phase = "sim_minus"
# set axis number
axis = 1

In [15]:
# load the skewers x 
meas_path = "/Users/mlokken/research/lyman_alpha/data/"
delta_x = np.load(meas_path+"snap_{:d}/delta_x/skewers_6_psim_minus_ax1_Ns768_wM0.05_sT1.0_sg1.0.npz".format(snap_num))
delta_flux_x = delta_x['delta_flux_x'] # these are already normalized
Np = delta_flux_x.shape[1]
L = delta_x['L_Mpc']
pix_spacing = L/Np
print("pixel spacing is", pix_spacing, "Mpc")
# xpar = delta_x['xpar'] # this is still wrong for some reason, redefined below
delta_x.close()
xpar = np.arange(0, L, pix_spacing)+pix_spacing/2.



pixel spacing is 0.05 Mpc


In [17]:
nskew = delta_flux_x.shape[0]
nside = np.sqrt(nskew).astype(int)
print(f"Box is {L} Mpc per side with {nside} skewers per side")
print(f"Spacing between pixels along line-of-sight = {pix_spacing} Mpc")

# bin delta_flux along line-of-sight
binsize = 10
if Np%binsize != 0:
    print("Np is not divisible by binsize")
    sys.exit()
delta_flux_b, xpar_b = est.bin_spectra(delta_flux_x, xpar, binsize)
Np_b=delta_flux_b.shape[1]
pix_spacing_b = xpar_b[1]-xpar_b[0]
print(f"Spacing between pixels along line-of-sight after binning = {pix_spacing_b} Mpc")
delta_flux_k = np.fft.rfft(delta_flux_b)*np.sqrt(pix_spacing_b/Np_b)
Nk = delta_flux_k.shape[1]
# get the positions
xpos = np.linspace(0,L,nside)
ypos = np.linspace(0,L,nside)
print("spacing between neighboring skewers is {:.2f} Mpc".format(xpos[1]-xpos[0]))
positions = np.array([[x,y] for x in xpos for y in ypos])
subsel=True
nsub=100
if subsel:
    randoms = rng.choice(len(positions), size=nsub)
    sel = np.zeros(len(positions), dtype=bool)
    sel[randoms] = True
else:
    sel = np.ones(len(positions), dtype=bool)
positions_sel = np.copy(positions)
positions_sel[~sel] = np.nan
delta_flux_x[~sel] = np.nan
delta_flux_b[~sel] = np.nan
delta_flux_k[~sel] = np.nan

# make masked arrays
delta_flux_x = np.ma.masked_invalid(delta_flux_x)
delta_flux_b = np.ma.masked_invalid(delta_flux_b)
delta_flux_k = np.ma.masked_invalid(delta_flux_k)
positions_sel = np.ma.masked_invalid(positions_sel)

# reshape into a grid
delta_flux_x_grid = np.reshape(delta_flux_x, (int(nside), int(nside), Np))
delta_flux_b_grid = np.reshape(delta_flux_b, (int(nside), int(nside), Np_b))
delta_flux_k_grid = np.reshape(delta_flux_k, (int(nside), int(nside), Nk))
position_grid = np.reshape(positions_sel, (int(nside), int(nside), 2))

delta_x_matrix_binned = est.make_delta_x_matrix(L, Np_b, xpar_b)

Box is 67.5 Mpc per side with 768 skewers per side
Spacing between pixels along line-of-sight = 0.05 Mpc
Number of bins: 135
Spacing between pixels along line-of-sight after binning = 0.49999999999999994 Mpc
spacing between neighboring skewers is 0.09 Mpc


In [18]:
# get the kpar that will correspond to the xpar
kpar_b = np.fft.rfftfreq(Np_b, pix_spacing_b)*2*np.pi # frequency in Mpc^-1
kpar   = np.fft.rfftfreq(Np, pix_spacing)*2*np.pi # frequency in Mpc^-1
kbin_est = [[kp-(kpar_b[1]-kpar_b[0])/2, kp+(kpar_b[1]-kpar_b[0])/2] for kp in kpar_b]


In [23]:
%%prun -s cumulative -q -l 10 -T prun0
kbin_centers, theta_est_p1d, F_alpha_beta_p1d, Lalpha_P1D = est.estimate_p1d(Np_b, delta_x_matrix_binned, pix_spacing_b, delta_flux_b_grid, kbin_est, np.zeros(delta_x_matrix_binned.shape), np.ones(delta_x_matrix_binned.shape))


Starting P1D.
center of kbins:  [0.0, 0.09308422677303092, 0.18616845354606185, 0.2792526803190928, 0.3723369070921237, 0.4654211338651546, 0.5585053606381856, 0.6515895874112164, 0.7446738141842474, 0.8377580409572783, 0.9308422677303092, 1.0239264945033402, 1.1170107212763711, 1.2100949480494019, 1.3031791748224328, 1.3962634015954638, 1.4893476283684948, 1.5824318551415255, 1.6755160819145567, 1.7686003086875874, 1.8616845354606184, 1.9547687622336496, 2.0478529890066803, 2.140937215779711, 2.2340214425527423, 2.327105669325773, 2.4201898960988038, 2.513274122871835, 2.6063583496448657, 2.699442576417897, 2.7925268031909276, 2.8856110299639584, 2.9786952567369895, 3.0717794835100207, 3.164863710283051, 3.257947937056082, 3.3510321638291134, 3.444116390602144, 3.537200617375175, 3.6302848441482056, 3.723369070921237, 3.816453297694268, 3.9095375244672987, 4.00262175124033, 4.095705978013361, 4.188790204786391, 4.281874431559422, 4.374958658332454, 4.4680428851054845, 4.56112711187851

In [ ]:
pk_avg = np.ma.mean((delta_flux_k.__abs__())**2, axis=0)


In [ ]:
plt.plot(kpar_b, pk_avg)
plt.plot(kbin_centers, theta_est_p1d)

In [24]:
print(open('prun0', 'r').read())


         4097680374 function calls (4097680170 primitive calls) in 1368.842 seconds

   Ordered by: cumulative time
   List reduced from 174 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000 1368.842 1368.842 {built-in method builtins.exec}
        1    0.001    0.001 1368.841 1368.841 <string>:1(<module>)
        1  198.100  198.100 1368.841 1368.841 estimator.py:314(estimate_p1d)
 80216064  170.241    0.000 1058.138    0.000 core.py:3205(__getitem__)
160520528  347.448    0.000  514.782    0.000 core.py:2940(_update_from)
160486528   38.954    0.000  485.709    0.000 {function MaskedArray.view at 0x10b472c00}
 80263664  102.436    0.000  446.832    0.000 core.py:2966(__array_finalize__)
 40108032   18.817    0.000  105.391    0.000 core.py:6654(is_masked)
1324469457   87.786    0.000   87.791    0.000 {built-in method builtins.getattr}
 40135232   11.903    0.000   75.827    0.000 {method 'any' of 'numpy